# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

In [8]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Inputs & Data

Explanation of each input and where it comes from.

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/nancy/user/riwata/projects/reward_comp_ext/results/2024_11_13_extract_lfp/proc/rce_pilot_2_standard_01_lfp_traces_and_frames.pkl")

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [10]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[489, 421, 500, 679, 635, 474, 382, 456, 562, ...","[236, 253, 386, 502, 398, 208, 115, 111, 51, -...","[315, 339, 420, 464, 367, 208, 133, 134, 89, 6...","[280, 279, 376, 442, 303, 102, -13, -22, -81, ...","[285, 407, 657, 874, 971, 959, 936, 955, 981, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[489, 421, 500, 679, 635, 474, 382, 456, 562, ...","[236, 253, 386, 502, 398, 208, 115, 111, 51, -...","[315, 339, 420, 464, 367, 208, 133, 134, 89, 6...","[280, 279, 376, 442, 303, 102, -13, -22, -81, ...","[285, 407, 657, 874, 971, 959, 936, 955, 981, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[102, 151, 170, 194, 233, 90, -173, -378, -460...","[151, 148, 133, 119, 110, 64, 4, -10, -27, -95...","[146, 130, 113, 117, 118, 71, 3, 3, 1, -96, -1...","[352, 463, 481, 367, 462, 639, 671, 673, 702, ...","[323, 445, 534, 446, 388, 465, 567, 532, 444, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[102, 151, 170, 194, 233, 90, -173, -378, -460...","[151, 148, 133, 119, 110, 64, 4, -10, -27, -95...","[146, 130, 113, 117, 118, 71, 3, 3, 1, -96, -1...","[352, 463, 481, 367, 462, 639, 671, 673, 702, ...","[323, 445, 534, 446, 388, 465, 567, 532, 444, ..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 24

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2_standard"

In [12]:
FULL_LFP_TRACES_PKL = "{}_02_lfp_traces_and_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [13]:
ALL_PHY_DIR

['/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-1_t1b2L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/p

In [14]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [15]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,8,0,0,1,9.198713,3,0,60.0,0.217516,mua,746,0,9
1,9,1,1,2,12.508228,3,0,60.0,1.605419,good,5506,0,10
2,19,1,0,2,13.435120,15,0,300.0,0.503261,good,1726,0,20
3,20,0,0,0,7.342390,20,0,400.0,0.219265,mua,752,0,22
4,21,47,10,122,10.664793,21,0,420.0,0.605604,noise,2077,0,23


- Combining all the unit info dataframes and adding the recording name

In [16]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [17]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230616_111904_standard_comp_to_training_D4_s...,0,1,1,9,21.699682,0,0.0,0.0,1.868411,noise,6304,0,1.0
1,20230616_111904_standard_comp_to_training_D4_s...,6,19,12,34,8.349835,2,0.0,40.0,3.400128,mua,11472,0,7.0
2,20230616_111904_standard_comp_to_training_D4_s...,7,2,1,10,13.197960,2,0.0,40.0,11.119177,good,37516,0,8.0
3,20230616_111904_standard_comp_to_training_D4_s...,9,27,10,79,8.396154,3,0.0,60.0,5.852713,mua,19747,0,10.0
4,20230616_111904_standard_comp_to_training_D4_s...,10,8,1,27,7.519016,3,0.0,60.0,2.798170,mua,9441,0,11.0


- Filtering for the good units

In [18]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [19]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230616_111904_standard_comp_to_training_D4_s...,7,2,1,10,13.197960,2,0.0,40.0,11.119177,good,37516,0,8.0
1,20230616_111904_standard_comp_to_training_D4_s...,23,1,0,3,9.657406,13,0.0,260.0,1.115889,good,3765,0,25.0
2,20230616_111904_standard_comp_to_training_D4_s...,30,0,0,1,17.081881,15,0.0,300.0,0.904567,good,3052,0,32.0
3,20230616_111904_standard_comp_to_training_D4_s...,53,0,0,0,19.220345,24,0.0,480.0,2.608780,good,8802,0,47.0
4,20230616_111904_standard_comp_to_training_D4_s...,64,1,0,3,9.148628,20,0.0,400.0,1.644344,good,5548,0,39.0


In [20]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [21]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 82,  33, 121, ...,  36,  35,  36], dtype=int32)

In [23]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(829550,)

- The times that all the spikes happened

In [24]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[      93],
       [     151],
       [     335],
       ...,
       [67479567],
       [67479676],
       [67479729]])

In [26]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(829550, 1)

### Combining everything into a dataframe

In [27]:
recording_to_spike_df = {}

for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [28]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,15,0,0,1,14.411364,6,0.0,120.0,0.901093,good,3263,0,17.0
1,17,0,0,1,10.738522,8,0.0,160.0,1.143280,good,4140,0,19.0
2,19,0,0,0,11.480671,8,0.0,160.0,1.276939,noise,4624,0,21.0
3,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0
4,35,6,2,23,7.762705,19,0.0,380.0,2.981642,mua,10797,0,37.0


In [29]:
spike_times

array([[      89],
       [     116],
       [     188],
       ...,
       [72422846],
       [72422932],
       [72423021]])

In [30]:
spike_clusters

array([141, 254,  32, ..., 141, 157,  32], dtype=int32)

- Combining the spike time df for all recordings

In [31]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [32]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [33]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,23,416,23,1,0,3,9.657406,13,0.0,260.0,1.115889,good,3765,0,25.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
1,80,1123,80,7,3,11,7.514018,2,NaN,40.0,5.136944,good,17332,0,NaN,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
2,53,1395,53,0,0,0,19.220345,24,0.0,480.0,2.608780,good,8802,0,47.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
3,102,1830,102,0,0,0,17.968143,10,0.0,200.0,1.499115,good,5058,0,22.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
4,30,2302,30,0,0,1,17.081881,15,0.0,300.0,0.904567,good,3052,0,32.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN


In [34]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
4630787,32,72421631,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,394.0,0.01970
4630788,32,72421974,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,343.0,0.01715
4630789,32,72422482,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,508.0,0.02540
4630790,141,72422846,141,29,15,62,7.976220,2,0.0,40.0,20.363095,good,73738,0,9.0,20230625_112913_standard_comp_to_both_rewarded...,3162.0,0.15810
4630791,32,72423021,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,539.0,0.02695


## Grouping all the neurons by recording

In [35]:
# Grouping all spike times by neuron and recording
grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: np.array(x)).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)

In [36]:
grouped_df.head()

,spike_clusters,recording_name,spike_times
0,13,20230616_111904_standard_comp_to_training_D4_s...,"[164503, 214157, 219948, 228496, 254211, 25605..."
1,43,20230616_111904_standard_comp_to_training_D4_s...,"[10518, 16788, 33219, 40252, 43753, 45536, 495..."
2,46,20230616_111904_standard_comp_to_training_D4_s...,"[79383, 181665, 237836, 301387, 327537, 351182..."
3,67,20230616_111904_standard_comp_to_training_D4_s...,"[71248, 72927, 78139, 79747, 80418, 81025, 813..."
4,68,20230616_111904_standard_comp_to_training_D4_s...,"[20982, 66688, 94142, 176325, 216673, 217233, ..."


In [37]:
max_number_of_spikes = all_spike_time_df["n_spikes"].max()

In [38]:
max_number_of_spikes

179245

In [39]:
grouped_df["spike_times"] = grouped_df["spike_times"].apply(lambda x: np.concatenate([x, np.full([max_number_of_spikes - x.shape[0]], np.nan)]))

In [40]:
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: np.array(list(x))}).reset_index()

In [41]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times
0,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31...","[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542..."
1,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31...","[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542..."
2,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ...","[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,..."
3,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ...","[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,..."


## Calculating the firing rates

In [43]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'recording_name', 'spike_clusters',
       'spike_times'],
      dtype='object')

In [44]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,15,0,0,1,14.411364,6,0.0,120.0,0.901093,good,3263,0,17.0
1,17,0,0,1,10.738522,8,0.0,160.0,1.143280,good,4140,0,19.0
2,19,0,0,0,11.480671,8,0.0,160.0,1.276939,noise,4624,0,21.0
3,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0
4,35,6,2,23,7.762705,19,0.0,380.0,2.981642,mua,10797,0,37.0


In [45]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times
0,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31...","[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542..."
1,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31...","[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542..."
2,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ...","[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,..."
3,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4,...,"[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ...","[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,..."


In [46]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times[~np.isnan(times)]), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)


In [47]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0][~np.isnan(x["spike_times"][0])], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [48]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [49]:
LFP_SPECTRAL_DF.iloc[:5,:10]

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject
0,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2
1,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,1.2
2,2,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4
3,2,20230616_111904_standard_comp_to_training_D4_s...,"[[109, 358], [3101, 3349], [5967, 6217], [8211...","[[422, 423], [425, 480], [480, 601], [869, 877...","[[36005, 36005], [36006, 36008], [36009, 36010...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.4,1.4


In [50]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace
0,"[1.2, 1.4]",307664,67788422,"[1383, 1383, 2769, 4155, 5541, 5541, 6927, 831...","[[88015, 288015], [2488011, 2688012], [4788011...","[[339014, 340215], [341012, 386212], [386615, ...","[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31..."
1,"[1.2, 1.4]",307664,67788422,"[1383, 1383, 2769, 2769, 4155, 5541, 5541, 692...","[[88015, 288015], [2488011, 2688012], [4788011...","[[339014, 340215], [341012, 386212], [386615, ...","[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-25, 131, 186, 119, 163, 162, 192, 217, 221, ...","[-62, 100, 155, 49, 89, 177, 289, 360, 379, 31..."
2,"[1.2, 1.4]",307664,67788422,"[1383, 1383, 2769, 4155, 5541, 5541, 6927, 831...","[[88015, 288015], [2488011, 2688012], [4788011...","[[339014, 340215], [341012, 386212], [386615, ...","[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ..."
3,"[1.2, 1.4]",307664,67788422,"[1383, 1383, 2769, 2769, 4155, 5541, 5541, 692...","[[88015, 288015], [2488011, 2688012], [4788011...","[[339014, 340215], [341012, 386212], [386615, ...","[[31772990, 31773790], [31774790, 31776190], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-84, -70, -5, 88, 158, 166, 109, 88, 99, 35, ...","[206, 322, 390, 404, 553, 604, 471, 312, 315, ..."


In [51]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[-65, 76, 83, -4, 109, 215, 271, 209, 116, 116...","[-164, -125, -125, -181, -210, -186, -128, -18...","[-574, -494, -433, -469, -539, -588, -558, -61...",20230616_111904_standard_comp_to_training_D4_s...,"[13, 43, 46, 67, 68, 120, 128, 133, 151, 193, ...","[[164503.0, 214157.0, 219948.0, 228496.0, 2542...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,"[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,"[9, 12, 47, 114, 212, 284, 236, 121, 87, 171, ...","[94, 178, 267, 320, 390, 378, 316, 318, 461, 6...","[116, 211, 284, 260, 293, 351, 330, 197, 123, ...",20230616_111904_standard_comp_to_training_D4_s...,"[7, 23, 30, 53, 64, 73, 80, 90, 100, 102, 104,...","[[15575.0, 17748.0, 19503.0, 22868.0, 29093.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."


In [52]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

""
0
1
2
3


In [53]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

(33740,)

In [54]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

(19, 33740)

In [55]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

17.0

In [56]:
LFP_SPECTRAL_DF.to_pickle(os.path.join("./proc", FULL_LFP_TRACES_PKL))

In [57]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'recording_name', 'spike_clusters',
       'spike_times', 'neuron_average_fr', 'neuron_average_timestamps'],
      dtype='object')

In [58]:
LFP_SPECTRAL_DF["spike_clusters"].iloc[0]

[13,
 43,
 46,
 67,
 68,
 120,
 128,
 133,
 151,
 193,
 205,
 209,
 210,
 214,
 218,
 222,
 229,
 243,
 249]

In [59]:
LFP_SPECTRAL_DF["spike_times"].iloc[0].shape

(19, 179245)

In [60]:
raise ValueError("")

ValueError: 